<a href="https://colab.research.google.com/github/jborders23/math6397-funhouse/blob/main/MATH6397_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#load data from GitHub
tesla = pd.read_csv("https://raw.githubusercontent.com/jborders23/math6397-funhouse/master/TSLA.csv")
ford = pd.read_csv("https://raw.githubusercontent.com/jborders23/math6397-funhouse/master/F.csv")
gm = pd.read_csv("https://raw.githubusercontent.com/jborders23/math6397-funhouse/master/GM.csv")

